#DATASCI W261: Machine Learning at Scale 

#Vineet Gangwar
**vineet.gangwar@gmail.com  
W261-2: Machine Learning at Scale  
Week1 - Quiz 1.12.2  
Date: Sep - 01 - 2015**

#This notebook provides a poor man Hadoop through command-line and python. Please insert the python code by yourself.

#Map
**Mapper Interface:**  
*Input:* The mapper takes a word and a filename as input  
*Ouput:* Writes a number into stdout

**Mapper function:**  
The mapper function implements the functionality of grep and wc. It takes a word and filename as inputs. The mapper then iterates over each line of the input file. It uses python re module's search method to search for one more occurrences of the word and increments a counter if it does. It then writes the counter into STDOUT.  
The mapper compiles the the regular expression before the loop which speeds up execution as the regular expression need not be compiled during every search.

In [20]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
count = 0
# Commenting out the below line
#WORD_RE = re.compile(r"[\w']+")
findword = sys.argv[1]
filename = sys.argv[2]
# Adding this line
WORD_RE = re.compile(findword, re.I)     # Compiling once. This speeds up execution as the mapper iterates
#
with open (filename, "r") as myfile:
#Please insert your code
    for line in myfile:
        result = WORD_RE.search(line)    # Search if findword exists in the line
        if result != None:               # Incrementing count if it does
            count += 1
print count                              # Writing to stdout

Overwriting mapper.py


In [14]:
!chmod a+x mapper.py

#Reduce
**Reducer Interface:**  
*Input:* A sequence of numbers separated by newline  
*Ouput:* A single number

**Reduce Function:**  
The reducer reads in a sequeunce of numbers from STDIN separated by a newline. These numbers were output by the mapper and each number corresponds to one chuck file. It then sums up the numbers and writes the sum into STDOUT.

In [15]:
%%writefile reducer.py
#!/usr/bin/python
import sys
sum = 0
for line in sys.stdin:
#Please insert your code
    sum += int(line)      # Adding up all the intermidiate counts
print sum                 # Writing to stdout

Overwriting reducer.py


In [16]:
!chmod a+x reducer.py

# Write script to file

In [17]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Overwriting pGrepCount.sh


#Run the file

In [18]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [19]:
!./pGrepCount.sh License.txt COPYRIGHT 4k

found [57] [COPYRIGHT] in the file [License.txt]
